# Geopandas

Geopandas provides a pandas-like syntax for geospatial operations.

## Setup

To work with both Python 2 and 3, this code requires the `future` module.
You can install it by running the following code block.

(Note: In Jupyter notebooks, code prefixed with `!` is interpreted as a shell command rather than Python code).

In [ ]:
!pip install --user future

Let's start by reading in some data.
Because it's familiar, we'll again use the US Census state boundaries and congressional districts.
The following code block will grab this data for you.

**EXERCISE:**
Run the following code block.
While it's running, annotate it with comments describing what each of the lines does.

In [ ]:
import os
import zipfile

# For Python 2-3 compatibility
from future.standard_library import install_aliases
install_aliases()

import urllib.request

data_dir = "data"
if (not os.path.exists(data_dir)):
    os.mkdir(data_dir)
    
state_url = "https://www2.census.gov/geo/tiger/TIGER2016/STATE/tl_2016_us_state.zip"
state_dir = os.path.join(data_dir, "tl_2016_us_state")
state_zip = "%s.zip" % state_dir
statefile = os.path.join(state_dir, "tl_2016_us_state.shp")

if (not os.path.exists(statefile)):
    if (~os.path.exists(state_zip)):
        urllib.request.urlretrieve(state_url, state_zip)
    zipfile.ZipFile(state_zip).extractall(state_dir)

cd_url = "https://www2.census.gov/geo/tiger/TIGER2016/CD/tl_2016_us_cd115.zip"
cd_dir = os.path.join(data_dir, "tl_2016_us_cd115")
cd_zip = "%s.zip" % cd_dir
cdfile = os.path.join(cd_dir, "tl_2016_us_cd115.shp")

if (not os.path.exists(cdfile)):
    if (~os.path.exists(cd_zip)):
        urllib.request.urlretrieve(cd_url, cd_zip)
    zipfile.ZipFile(cd_zip).extractall(cd_dir)

Now, let's load geopandas and read our first file.

In [ ]:
import geopandas as gpd

states = gpd.read_file(statefile)

In [ ]:
states

Even though it read a shapefile, the underlying `states` object is a data frame, just like the ones you've worked with previously.
It supports all of the same Pandas operations you are already used to:

In [ ]:
states.head()

In [ ]:
states.info()

(Quick aside: If you're curious _why_ string columns have type object, [here](https://stackoverflow.com/a/21020411/2477097) is a concise answer from Stack Overflow).

# Pandas review: Working with columns

### Selecting columns

Pandas offers many ways to select individual columns.


(1) Passing the column name as a string (note the quotes!).

In [ ]:
states["NAME"]

(2) As an _attribute_ of the data frame object, via the `object.attribute` syntax

In [ ]:
states.NAME

---
**EXERCISE**:
Select the column that stores the state abbreviations.
Do this twice --- once with each method.

In [ ]:
# Answer here...

**QUESTION**:
What are some advantages of method 1?
What are some advantages of method 2?
Which method do you prefer (right now) and why?

{Answer here}

---
By replacing method 1's string with a `list`, you can select multiple columns by name.

In [ ]:
states[["NAME", "ALAND"]]

Recall that strings and lists, like all objects in Python, can be assigned to variables.
So, the above could be achieved with code like the following (pay attention to which code is and is not quoted):

In [ ]:
name_col = "NAME"
states[name_col]

In [ ]:
my_cols = [name_col, "ALAND", "AWATER"]
print my_cols

In [ ]:
states[my_cols]

---
**EXERCISE**:
Use this syntax to create a table of state names and abbreviations.

In [ ]:
# Answer here

### Creating new columns

First, let's create a "sandbox" data frame that we can safely modify by using the `DataFrame.copy` method.

In [ ]:
states2 = states.copy()

> WEEDS: The reason we can't just do something like `states2 = states` is that Pandas does a "shallow copy" by default, which means that rather than copying the entire `states` object, this just creates a new reference to the same object.
> In other words, if we just did `states2 = states`, any changes we made to `states2` _would also apply to_ `states`, because both of those variables point to the same object in the computer's memory.
> This is actually pretty useful for memory efficiency, but is a bit counterintuitive and counter to our immediate aims here---hence, `states2 = states.copy()`.

You can use similar syntax for selecting columns (Method 1) to create new ones.

In [ ]:
states2["dummy_column"] = 42
states2.info()

However, note that you _cannot_ create new columns by using the `object.attribute` syntax.
Confusingly, this *will not throw an error*, but will silently do nothing.*

> \* Technically, it creates a new attribute for the object that you will be able to access and do things with. But this is pretty non-standard practice, and if you're not careful, you could potentially accidentally overwrite important methods that prevent the data frame from working correctly. So I would avoid this. 

Note that although we passed only one value to the data frame, it was automatically "recycled" to every row.

In [ ]:
states2["dummy_column"].head()

A common use case for creating new columns is doing math on existing columns.
For example, let's convert the state land areas from square meters to hectares (1 ha = 100 m x 100 m = 10,000 m$^2$).

In [ ]:
# Note the use of 10000.0, not just 10000, to avoid integer math
states2["ALAND_ha"] = states2["ALAND"] / 10000.0
states2[["NAME", "ALAND", "ALAND_ha"]].head()

(In case you haven't seen it before, `e+06` is scientific notation, i.e. `6.22e+06` means $6.22 \times 10^6$).

---
**EXERCISE**:
Create a column `TOTAL_AREA` that is the sum of land and water area for each state.

In [ ]:
# Answer here

---
Column assignment can also be used to modify existing columns in place.

The `REGION` and `DIVISION` columns are stored as strings, but since they are all integers, let's convert them to make our lives easier.

In [ ]:
states2[["REGION", "DIVISION"]].head()

To convert columns to a different type, us the `astype()` method.

One way to do this is for multiple columns is with a `for` loop:

In [ ]:
int_cols = ["REGION", "DIVISION"]
for col in int_cols:
    states2[col] = states2[col].astype(int)
states2.info()

Fortunately, Pandas also allows this function to work on multiple columns in one go:

In [ ]:
states2[int_cols] = states2[int_cols].astype(int)
states2.info()

**QUESTION**: The `STATEFP` and `STATEENS` columns are also digits that are stored as strings.
Why might it be a good idea to keep them as strings?
(HINT: Consider ZIP codes. Look up the ZIP code for Brookline, MA, and try to store it as an integer. What happens?) 

{Your answer here}

**QUESTION**:
Why might `float` be a better type for the `ALAND` and `AWATER` columns?

{Your answer here}

**EXERCISE**:
Convert these two columns to `float`.
Use the `info()` method to confirm that your code worked.

In [ ]:
# Your answer here

**EXERCISE**:
Create new columns `FRAC_LAND` and `FRAC_WATER` containing the area fractions of land and water, respectivly, for each state.

## Pandas review: Working with rows

Somewhat confusingly, the `[]` operators in Pandas can be used to select rows, but _only_ if they are passed _slices_.

In [ ]:
states[0:4]

Again, this _only_ works for _slices_ defined with the `start:end` syntax.
The following will not work:

In [ ]:
states[1]

To select individual rows this way, you will have to be clever about creating slices containing only one number.

In [ ]:
states[3:4]

To review, the `head(n)` and `tail(n)` methods can be used to select the first and last `n` rows of a data frame.

In [ ]:
states.head(3)

In [ ]:
states.tail(4)

A more useful way to select rows is based on values of specific columns.

First, let's see how comparison operators (`==`, `>`, etc.) work on columns.

In [ ]:
states["NAME"] == "West Virginia"

Note that this returns a `Series` (Pandas class for a column) of type `bool` (boolean), with value `True` where the condition is met (only in the first row) and `False` everywhere else.

We can then use this series for subsetting.

In [ ]:
wv = states["NAME"] == "West Virginia"
states[wv]

You can combine multiple conditions using the `&` ("bitwise and") or `|` ("bitwise or") operators (but, note that each statement has to be wrapped in parentheses, `()`).

For example, this selects some really small states (area < $10^9 \text{m}^2$).

In [ ]:
states[states["ALAND"] < 10e9]

This selects really small states that are also in region 3 (again, pay attention to the parentheses).

In [ ]:
states[(states["ALAND"] < 10e9) & (states["REGION"] == "3")]

Meanwhile, this selects some really small states and some really large ones (area less than $10^9 \text{m}^2$ or greater than $10^{12} \text{m}^2$).

In [ ]:
states[(states["ALAND"] < 1e9) | (states["ALAND"] > 1e12)]

As you can see, "bitwise" here means that these operators will text the conditional statement element by element. For example:
```
(True True False) &
(True False True)
------------------
(True False False)

(True True False) |
(True False True)
------------------
(True True True)
```

These should not be confused with the logical operators `and` and `or`, which always return a single `True` or `False` and are therefore more useful in `if-else` statements.
For more information, see [this Stack Overflow post](https://stackoverflow.com/questions/22646463/and-boolean-vs-bitwise-why-difference-in-behavior-with-lists-vs-nump).

Another useful operation is the `isin` method for selecting values that are members of a specific set.

In [ ]:
my_subset = ["WV", "AK", "OK"]
states[states["STUSPS"].isin(my_subset)]

Finally, any of these conditions can be negated with the `~` ("bitwise not") operator.
This "flips" every logical element to its opposite value.
For example:

```
~(True True False) --> (False False True)
```

For example, the following code selects every state _except_ West Virginia and Florida.

In [ ]:
states[~(states.STUSPS.isin(["WV", "FL"]))]

---
**EXERCISE**:
Select states whose land area is between $10^9$ and $10^{10}$ square meters.

In [ ]:
# Answer here

**EXERCISE**:
Select states in regions 1 and 2.

In [ ]:
# Answer

**EXERCISE**: Select every state that is _not_ in region 9.

In [ ]:
# Answer

---

## Geopandas: Basics and simple maps

Let's look more closely at the structure of the `states` `DataFrame`.

In [ ]:
states.info()

Note that the last column is called `geometry` and has type `geometry`.
This column stores the spatial information used by GIS software and analysis tools.

In [ ]:
states["geometry"].head()

We are dealing with vector data, so the geometry information consists of the vector type (e.g. Polygon, MultiPolygon, Line, Point) and its corresponding coordinate pairs.

Let's see what happens if we try to plot this.

In [ ]:
# NOTE: You may have to run this code block twice to get the map to appear.
states.plot()

It's not a very good one, but it's a map of the US states!

Let's see if we can zoom in on a few states using the syntax we used earlier.

In [ ]:
states[states["NAME"] == "West Virginia"].plot()

---
**EXERCISE**:
Plot the states in region 1.

In [ ]:
# Answer here

---

### Thematic maps

Geopandas makes it really easy to make thematic maps.
Simply pass a column name to the `column` argument of `plot()`.

In [ ]:
states[states.REGION.isin([1,2])].plot(column = "DIVISION")

For our current purposes, these crude, simple maps will suffice.
You can find more information on adjusting map aesthetics in the [geopandas manual](http://geopandas.org/mapping.html).

**EXERCISE**:
Figure out which `REGION` Texas is in.
Then, generate a map of that region with each state having its own color.

In [ ]:
# Answer here

## Spatial joins

Spatial joins combine information in two datasets based on their spatial relationships.
A useful application for us is to combine our data on state boundaries with our data on congressional districts.

First, let's read in the congressional district data.

In [ ]:
cd116 = gpd.read_file(cdfile)
cd116.info()

In [ ]:
cd116[["NAMELSAD", "geometry"]].head()

In [ ]:
states[["NAME", "geometry"]].head()

To combine these two, we use the `geopandas.sjoin` method.
This will take two geometries and the kind of spatial relationship by which they will be joined.
In this case, we expect that congressional districts will be `"within"` their corresponding states, which means, formally, that one object's boundaries intersect the interior of the other object.

In [ ]:
districts = gpd.sjoin(cd116, states, op = "within")

Now, let's inspect the resulting object.

In [ ]:
districts.info()

In [ ]:
districts.sort_values(["NAME", "NAMELSAD"])

The resulting `districts` data frame now has the attributes of the `states` and `cd116` data frames combined together.
This means that we can now operate on them together.

For example, let's plot the congressional districts in Alabama.

In [ ]:
districts[districts.NAME == "Alabama"].plot(column = "NAMELSAD")

---
**EXERCISE**:
Plot the congressional districts of your home state and its two neighbors.

In [ ]:
# Answer here

### Exporting shapefiles

Now, we can use our new data frame to create shapefiles for individual states.
The method for doing this is `object.to_file()`.
Geopandas is smart enough to figure out the type of object to export from the file extension.

In [ ]:
mystate = "Arkansas"
result = districts[districts.NAME == mystate]
result.head()

To stay organized, let's create an `"output"` directory and store the results there.

In [ ]:
outdir = "output"
if (not os.path.exists(outdir)):
    os.mkdir(outdir)
alabama.to_file(os.path.join(outdir, "alabama.shp"))

---
**EXERCISE**:
Use a `for` loop to export the shapefiles of 5 states of your choice.

### Attribute joins

The previous example involved a _spatial join_, in which we matched two different datasets based on a spatial relationship.
Another kind of join is an _attribute join_, which, as its name suggests, combines two different datasets based on one or more overlapping attributes (columns).

Consider the following conceptual example.
(Aside: Notice this syntax for creating `DataFrames` using `dict`s).

In [ ]:
import pandas as pd

food_types = pd.DataFrame({
    "food_type" : ["fruit", "fruit", "vegetable"],
    "food" : ["apple", "orange", "cucumber"]
})

food_prices = pd.DataFrame({
    "food" : ["cucumber", "orange", "apple"],
    "price" : [1.75, 1.2, 0.89]
})

print("Food types:")
print(food_types)
print("Food prices:")
print(food_prices)

Suppose we wanted to calculate average food price by food type.
To do that, we need to combine these two data frames based on something they have in common -- in this case, the `food` column.
This can be accomplished via the `merge()` method.

In [ ]:
food_both = food_types.merge(food_prices, on="food")
food_both

The `merge` function takes two required arguments---the two data frames being combined---and an optional `on=` arugment that specifies which column(s) the two data frames have in common.
Now that the data are combined, we can calculate the average price by combining `groupby` and `mean`. 

In [ ]:
food_both.groupby("food_type").mean()

Now, let's apply this principle to real data.
Following with our theme about states and electoral districts, we will use the statewide election results from the [MIT Election Data lab](https://electionlab.mit.edu/data).
Again, we can use some Python code to download this data automatically.

In [ ]:
election_file = os.path.join(data_dir, "president-bystate.csv")

if not os.path.exists(election_file):
    election_url = "https://dataverse.harvard.edu/api/access/datafile/3444051?format=original&gbrecs=true"
    urllib.request.urlretrieve(election_url, election_file)

election_full = pd.read_csv(election_file)
election_full.head()

In [ ]:
election_full.shape

This dataset contains the vote counts in every presidential election from 1976 to 2016 by state and candidate.

---
**EXERCISE**:
Use your pandas subsetting skills to answer the following questions.
Show your work!

(1) Who won California in the 1984 election?

In [ ]:
# Answer here

(2) What was the largest ever vote _fraction_ won by a third-party candidate?
In what year and state state did that in?

In [ ]:
# Answer here

---

Now that we have some familiarity with this dataset, let's use it to create an election map by combining it with our current state shapefile.
First, let's identify which columns we will use for the merge.

In [ ]:
election_full.sort_values("state").head()

In [ ]:
states.sort_values("NAME").head()

Looks like we have several candidate attributes that we can use for the join.
For the purposes of this exercise, let's use the state two-letter code ("AZ", "TX", etc.).
However, we have a problem: this attribute is named different things in the two datasets---`STUSPS` in `states` and `state_po` in `election_full`.
Fortunately, this is a common use case that `merge` is well-equipped to handle;
we just have to replace the use of `on` with more specific `left_on` (corresponding to the first argument) and `right_on` (the second argument): 

In [ ]:
election_geo = states.merge(
    election_full,
    left_on = "STUSPS",
    right_on = "state_po"
)

In [ ]:
election_geo.head()

Et voila! We now have a combined election data frame.
Let's use this to generate a map of state voting patterns.

> For now, don't worry about totally understanding the more involved plotting code here. We will cover this in more detail later.

In [ ]:
import matplotlib.pyplot as plt

# Set up two "subplots" within the figure:
# One for for the map, and one for the legend.
# This creates two objects, `fig` and `ax`.
fig, ax = plt.subplots(1, 1)

# This restricts the x-axis from -180 to -60 longitude.
# Otherwise, the Aleutian islands end up on the far right of the plot.
plt.xlim(-180, -60)

# This draws the plot itself
(election_geo[
    (election_geo.year == 2016) &
    (election_geo.party == "republican")]
 .plot(ax = ax, column = "candidatevotes", legend = True))
# `ax = ax` tells this to use the axes we've configured.
# `legend = True` says to also plot the legend.

This is not a particularly useful map because it's [just a map of population](https://xkcd.com/1138/).
A better map would show the _fraction_ of the republican vote.

In [ ]:
election_geo["candidate_frac"] = election_geo.candidatevotes / election_geo.totalvotes

fig, ax = plt.subplots(1, 1)
plt.xlim(-180, -60)
(election_geo[
    (election_geo.year == 2016) &
    (election_geo.party == "republican")]
 .plot(ax = ax, column = "candidate_frac", legend = True))

That's a bit better!
However, the continuous color gradient makes it difficult to pick out who won the election in any given state.

Doing this in Pandas is a bit complicated, so let's work through a contrived example.

In [ ]:
test_df = pd.DataFrame({
    "year" : [1997, 1997, 1998, 1998, 1999, 1999],
    "value" : [1.0, 1.3, 1.6, 1.9, 1.4, 1.5]
})
test_df

First, we introduce the `idxmax()`, which returns the index corresponding to the maximum value of a data frame.

In [ ]:
test_df.idxmax()

This is saying that the maximum value of the `value` column occurs in index 3 (the 4th row), while the maximum value of year occurs in index 4 (the 5th row).
(Note that in case of a tie, as we have here, it defaults to the lowest index).

We can combine this with `groupby()` to get the index of the largest value _within each group_.

In [ ]:
test_df.groupby("year").idxmax()

This is saying that the maximum `value` for year 1997 is in index 1 (2nd row), for year 1998 in index 3 (4th row), and for year 1999 in index 5 (6th row).

This is still a data frame, but what we really want is just the row indices corresponding to the values we want to maximize.
So let's pull that out.

In [ ]:
test_df.groupby("year").idxmax()["value"]

This gives us a `Series`, which can be used for indexing.
Finally, to get the values themselves, we need a way to retrieve rows given multiple indices.
This is what the `loc` attribute does.
(Note the use of hard brackets below).

In [ ]:
test_df.loc[[1,2,4]]

We now have everything we need, so let's put it all together!

In [ ]:
test_df.loc[test_df.groupby("year").idxmax()["value"]]

---
**EXERCISE**:
Use this template to retrieve the election winners by state and year from our election data.
(Replace the `?????` with the correct code.

In [ ]:
# Answer
election_winners = election_geo.?????[
    election_geo.?????(["??????", "?????"]).?????()["?????"]
]
election_winners.info()

---

In [ ]:
election_winners = election_geo.loc[
    election_geo.groupby(["year", "STATEFP"]).idxmax()["candidatevotes"]
]
election_winners.head()

Now, let's use this result to generate a map of the 2016 election.

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.xlim(-180, -60)

(election_winners[election_geo.year == 2016].plot(ax = ax, column = "party", legend = True))

---
**EXERCISE**:

(1) Generate a plot of the 2012 election, but assign the labels to the name of the candidate (rather than the party).

In [ ]:
# Answer here

(2) Save the `election_winners` data frame to a shapefile called `output/election_winners.shp`. 

In [ ]:
# Answer here